In [2]:
# import packages
import numpy as np

class NeuralNetwork:
    def __init__(self, layers, learnRate=0.50):
        #list of layers
        self.layers = layers
        #the learning rate
        self.learnRate = learnRate
        #list of weights between 1 and -1
        self.weights = []
        #errors at the last fit call
        self.errors = np.ones(16)
        
        #loop through the layers (except the 2 last one)
        for i in np.arange(0, len(layers) - 2):
            # init weight matrix and add a random bias
            w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
            self.weights.append(w / np.sqrt(layers[i]))
        # not adding a bias to the output layer
        w = np.random.randn(layers[-2] + 1, layers[-1])
        self.weights.append(w / np.sqrt(layers[-2]))
    
    def sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))

    def sigmoidDeriv(self, x):
        return x * (1 - x)
    
    #Check if all the errors form the pattern are all < errorMin
    def checkErrors(self, errorMin=0.05):
        error = 0
        for el in range (len(self.errors)):
            if self.errors[el] > error:
                error = self.errors[el]
        if error < 0.05:
            return True, error
        else:
            return False, error 
    
    def fit(self, X, y, epochsMax=10000, displayUpdate=100):
        
        #add a col of 1's to make the bias trainable
        X = np.c_[X, np.ones((X.shape[0]))]
        # main loop 
        cti = 0
        for epoch in np.arange(0, epochsMax):
            breaker = self.checkErrors()
            if(breaker[0] == True):
                print("Stopped at {} epoch, error is = {}".format(epoch + 1, breaker[1]))
                break
            # loop to train every datapoint
            for (x, target) in zip(X, y):
                self.fit_partial(x, target, cti)
                cti = cti + 1
                if cti == 16:
                    cti = 0
            # display info every displayUpdate
            if epoch == 0 or (epoch + 1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={:.7f}".format(
                    epoch + 1, loss))
    def fit_partial(self, x, y, cti):
        
        #list of output activation by layer
        A = [np.atleast_2d(x)]
        
        # loop over the layers in the network
        for layer in np.arange(0, len(self.weights)):
            
            # sigmoid function call over . of the matrixes
            net = A[layer].dot(self.weights[layer])
            out = self.sigmoid(net)
            # once we have the net output, add it to our list of
            # activations
            A.append(out)
            
        # BACKPROPAGATION
        # the first phase of backpropagation is to compute the
        # difference between our *prediction* (the final output
        # activation in the activations list) and the true target
        # value
        
        error = A[-1] - y
        #add the errors in the attribute
        self.errors[cti] = error
        
        # from here, we need to apply the chain rule and build our
        # list of deltas 'D'; the first entry in the deltas is
        # simply the error of the output layer times the derivative
        # of our activation function for the output value
        D = [error * self.sigmoidDeriv(A[-1])]
        
        # once you understand the chain rule it becomes super easy
        # to implement with a 'for' loop -- simply loop over the
        # layers in reverse order (ignoring the last two since we
        # already have taken them into account)
        for layer in np.arange(len(A) - 2, 0, -1):
            # the delta for the current layer is equal to the delta
            # of the *previous layer* dotted with the weight matrix
            # of the current layer, followed by multiplying the delta
            # by the derivative of the nonlinear activation function
            # for the activations of the current layer
            delta = D[-1].dot(self.weights[layer].T)
            delta = delta * self.sigmoidDeriv(A[layer])
            D.append(delta)
        
        # since we looped over our layers in reverse order we need to
        # reverse the deltas
        D = D[::-1]
        # WEIGHT UPDATE PHASE
        # loop over the layers
        for layer in np.arange(0, len(self.weights)):
            # update our weights by taking the dot product of the layer
            # activations with their respective deltas, then multiplying
            # this value by some small learning rate and adding to our
            # weight matrix -- this is where the actual "learning" takes
            # place
            self.weights[layer] += -self.learnRate * A[layer].T.dot(D[layer])
            
    def predict(self, X, addBias=True):
            # initialize the output prediction as the input features -- this
            # value will be (forward) propagated through the network to
            # obtain the final prediction
            p = np.atleast_2d(X)
            # check to see if the bias column should be added
            if addBias:
                # insert a column of 1's as the last entry in the feature
                # matrix (bias)
                p = np.c_[p, np.ones((p.shape[0]))]
            # loop over our layers in the network
            for layer in np.arange(0, len(self.weights)):
                # computing the output prediction is as simple as taking
                # the dot product between the current activation value 'p'
                # and the weight matrix associated with the current layer,
                # then passing this value through a nonlinear activation
                # function
                p = self.sigmoid(np.dot(p, self.weights[layer]))
            # return the predicted value
            return p
    
    def calculate_loss(self, X, targets):
        # make predictions for the input data points then compute
        # the loss
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        # return the loss
        return loss
    
    def __repr__(self):
        # toString Equivalent
        return "NeuralNetwork: {}".format(
            "-".join(str(l) for l in self.layers))
    
    def printW(self):
        print(self.weights)
        
    def printErr(self):
        print(self.errors) 

In [3]:
X = [[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

y = [0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]

In [12]:
nn = NeuralNetwork([4, 4, 1])
nn.fit(np.array(X), np.array(y), epochsMax=50000)
nn.printErr()
print(nn)

[INFO] epoch=1, loss=31.4242979
[INFO] epoch=100, loss=37.9696296
[INFO] epoch=200, loss=39.9459553
[INFO] epoch=300, loss=43.2414526
[INFO] epoch=400, loss=46.4228692
[INFO] epoch=500, loss=48.4535160
[INFO] epoch=600, loss=49.8061884
[INFO] epoch=700, loss=51.5763466
[INFO] epoch=800, loss=53.0066685
[INFO] epoch=900, loss=53.9690547
[INFO] epoch=1000, loss=54.6434304
[INFO] epoch=1100, loss=55.1441876
[INFO] epoch=1200, loss=55.5334442
[INFO] epoch=1300, loss=55.8466635
[INFO] epoch=1400, loss=56.1054609
[INFO] epoch=1500, loss=56.3237812
[INFO] epoch=1600, loss=56.5110538
[INFO] epoch=1700, loss=56.6739068
[INFO] epoch=1800, loss=56.8171510
[INFO] epoch=1900, loss=56.9443734
[INFO] epoch=2000, loss=57.0583096
[INFO] epoch=2100, loss=57.1610867
[INFO] epoch=2200, loss=57.2543869
[INFO] epoch=2300, loss=57.3395598
[INFO] epoch=2400, loss=57.4177023
[INFO] epoch=2500, loss=57.4897161
[INFO] epoch=2600, loss=57.5563502
[INFO] epoch=2700, loss=57.6182322
Stopped at 2731 epoch, error is 

In [14]:
for (x, target) in zip(X, y):
    #Checking prediction and printing
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    if (target == step):
        print("[INFO] data={}, expected={}, pred={:.4f}, step={}".format(
            x, target, pred, step))
    else:
        print("[ERR] data={}, expected={}, pred={:.4f}, step={}".format(
            x, target, pred, step))
    
        

[INFO] data=[0, 0, 0, 0], expected=0, pred=0.0112, step=0
[INFO] data=[0, 0, 0, 1], expected=0, pred=0.0268, step=0
[INFO] data=[0, 0, 1, 0], expected=0, pred=0.0138, step=0
[INFO] data=[0, 0, 1, 1], expected=1, pred=0.9761, step=1
[INFO] data=[0, 1, 0, 0], expected=0, pred=0.0105, step=0
[INFO] data=[0, 1, 0, 1], expected=1, pred=0.9514, step=1
[INFO] data=[0, 1, 1, 0], expected=1, pred=0.9719, step=1
[INFO] data=[0, 1, 1, 1], expected=0, pred=0.0498, step=0
[INFO] data=[1, 0, 0, 0], expected=0, pred=0.0000, step=0
[INFO] data=[1, 0, 0, 1], expected=1, pred=0.9791, step=1
[INFO] data=[1, 0, 1, 0], expected=0, pred=0.0132, step=0
[INFO] data=[1, 0, 1, 1], expected=1, pred=0.9946, step=1
[INFO] data=[1, 1, 0, 0], expected=0, pred=0.0084, step=0
[INFO] data=[1, 1, 0, 1], expected=0, pred=0.0212, step=0
[INFO] data=[1, 1, 1, 0], expected=0, pred=0.0119, step=0
[INFO] data=[1, 1, 1, 1], expected=1, pred=0.9835, step=1
